# Tutorial: Tool Use to Agentic Task (Search + Summary)

Goals:
- Demonstrate a tool pipeline from search to evidence retrieval to summary synthesis.
- Use a verifier to evaluate the quality of the final step.
- Run the same workflow in both `codeact` and `react` modes.

## Outline
1. Setup tools and helper functions.
2. Register search/summary tools.
3. Run agentic task `search + summary`.
4. Inspect tool trace and verifier decision.


In [ ]:
from __future__ import annotations

import json
import sys
from pathlib import Path
from urllib.parse import quote

import requests


def find_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / "pyproject.toml").exists() and (candidate / "src").exists():
            return candidate
    raise RuntimeError("Could not locate repo root")


REPO_ROOT = find_repo_root(Path.cwd())
if str(REPO_ROOT / "src") not in sys.path:
    sys.path.insert(0, str(REPO_ROOT / "src"))

from manus_three_agent.agents import CriticAgent
from manus_three_agent.core.types import ModelConfig
from manus_three_agent.prompts import PromptTemplates
from manus_three_agent.tools import build_default_tool_registry
from manus_three_agent.tracing import TraceCollector

prompts = PromptTemplates(config_dir=str(REPO_ROOT / "configs" / "prompts"))
model_cfg = ModelConfig(model="mock")
tracer = TraceCollector.disabled()

print(f"Repo root: {REPO_ROOT}")


## Step 1: Register search and summary tools

In [ ]:
WIKI_HEADERS = {
    "User-Agent": "manus-3subagent-repro/0.1 (education notebook; contact: dung.vpt@uney.com)"
}


def wiki_search_tool(arguments: dict) -> dict:
    query = str(arguments.get("query", "")).strip()
    max_results = int(arguments.get("max_results", 5))
    if not query:
        return {"ok": False, "error": "empty_query", "results": []}

    try:
        response = requests.get(
            "https://en.wikipedia.org/w/api.php",
            params={
                "action": "query",
                "list": "search",
                "format": "json",
                "srsearch": query,
                "srlimit": max_results,
            },
            headers=WIKI_HEADERS,
            timeout=20,
        )
        response.raise_for_status()
        data = response.json()
        items = data.get("query", {}).get("search", [])
        results = [
            {
                "title": item.get("title", ""),
                "snippet": item.get("snippet", ""),
                "pageid": item.get("pageid"),
            }
            for item in items
        ]
        return {"ok": True, "query": query, "results": results}
    except Exception as exc:
        return {"ok": False, "query": query, "error": f"{type(exc).__name__}: {exc}", "results": []}


def wiki_summary_tool(arguments: dict) -> dict:
    title = str(arguments.get("title", "")).strip()
    max_chars = int(arguments.get("max_chars", 700))
    if not title:
        return {"ok": False, "error": "empty_title", "title": ""}

    try:
        response = requests.get(
            f"https://en.wikipedia.org/api/rest_v1/page/summary/{quote(title)}",
            headers=WIKI_HEADERS,
            timeout=20,
        )
        if response.status_code >= 400:
            return {"ok": False, "title": title, "error": f"http_{response.status_code}"}

        data = response.json()
        extract = str(data.get("extract", ""))[:max_chars]
        return {"ok": True, "title": title, "extract": extract, "url": data.get("content_urls", {}).get("desktop", {}).get("page", "")}
    except Exception as exc:
        return {"ok": False, "title": title, "error": f"{type(exc).__name__}: {exc}"}


registry = build_default_tool_registry()
registry.register("wiki_search", wiki_search_tool)
registry.register("wiki_summary", wiki_summary_tool)

print("Registered tools: calculator, fetch_url, wiki_search, wiki_summary")


## Step 2: Define agentic task (search + summary)

In [ ]:
def synthesize_summaries(rows: list[dict], max_points: int = 4) -> str:
    bullets = []
    for row in rows[:max_points]:
        title = row.get("title", "")
        extract = row.get("extract", "")
        sentence = extract.split(".")[0].strip()
        if sentence:
            bullets.append(f"- {title}: {sentence}.")
    return "\n".join(bullets) if bullets else "No reliable summary could be produced."


def run_search_summary_agent(query: str, agentic_mode: str = "codeact", top_k: int = 3) -> dict:
    verifier = CriticAgent(model_cfg, prompts, tracer=tracer, force_mock=True, agentic_mode=agentic_mode)

    trace = []

    search_result = registry.call("wiki_search", {"query": query, "max_results": top_k})
    trace.append({"step": 1, "tool": "wiki_search", "ok": search_result.get("ok")})

    titles = [r.get("title", "") for r in search_result.get("output", {}).get("results", []) if r.get("title")]
    summary_rows = []

    for title in titles:
        summary_result = registry.call("wiki_summary", {"title": title, "max_chars": 700})
        trace.append({"step": 2, "tool": "wiki_summary", "title": title, "ok": summary_result.get("ok")})
        payload = summary_result.get("output", {})
        if payload.get("ok"):
            summary_rows.append(payload)

    synthesized = synthesize_summaries(summary_rows)

    verifier_review = verifier.review(
        goal=f"Search and summarize: {query}",
        observation=("success" if summary_rows else "failed") + " summary collection",
        action_history=[{"tool_trace_len": len(trace), "mode": agentic_mode}],
        current_step_idx=2,
        plan_length=2,
        step=2,
    )

    return {
        "query": query,
        "agentic_mode": agentic_mode,
        "titles": titles,
        "synthesized_summary": synthesized,
        "verifier": verifier_review.model_dump(),
        "trace": trace,
        "summary_rows": summary_rows,
    }


## Step 3: Run task and inspect outputs

In [ ]:
query = "Large language model agent"
results = {}
for mode in ["codeact", "react"]:
    results[mode] = run_search_summary_agent(query=query, agentic_mode=mode, top_k=3)

print(json.dumps({m: {"titles": v["titles"], "verifier": v["verifier"]} for m, v in results.items()}, indent=2, ensure_ascii=False))
print("\n--- Synthesized summary (codeact) ---\n")
print(results["codeact"]["synthesized_summary"])
print("\n--- Tool trace (codeact) ---\n")
print(json.dumps(results["codeact"]["trace"], indent=2, ensure_ascii=False))


## Step 4: Also demo built-in tools (`fetch_url`, `calculator`)

In [ ]:
calc = registry.call("calculator", {"expression": "(42 * 13) / 7"})
fetch = registry.call("fetch_url", {"url": "https://manus.im/blog", "max_chars": 400})

print(json.dumps({"calculator": calc, "fetch_url": {"ok": fetch.get("ok"), "title": fetch.get("output", {}).get("title")}}, indent=2, ensure_ascii=False))


## Takeaways
- A simple tool pipeline can be elevated into an agentic task with a verifier gate.
- `agentic_mode` allows one runtime to support different strategy profiles.
- Tool-step traces are high-value inputs for Worker/Verifier tuning datasets.